In [2]:
import requests
import pandas as pd
import json

# Movie ids coming from http://files.tmdb.org/p/exports/movie_ids_10_20_2020.json.gz
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

%matplotlib inline

try:
    fp = open('api_key.txt')
    api_key = fp.readline()
finally:
    fp.close()

In [3]:
tmp = pd.read_json("movie_ids_07_23_2021.json", lines=True)
all_movies = tmp.sort_values('popularity', ascending=False).reset_index(drop=True).head(25000).copy()
#del(tmp)

In [4]:
test = tmp.sort_values('popularity', ascending=False).reset_index(drop=True)
test[test.original_title=='The Love Guru']

,adult,id,original_title,popularity,video
8513,False,12177,The Love Guru,9.369,False


In [5]:
test

,adult,id,original_title,popularity,video
0,False,379686,Space Jam: A New Legacy,9427.563,False
1,False,602223,The Forever Purge,6464.587,False
2,False,497698,Black Widow,6296.895,False
3,False,459151,The Boss Baby: Family Business,4033.702,False
4,False,615658,Awake,3670.817,False
...,...,...,...,...,...
624831,False,853306,Je ne sais pas,0.000,False
624832,False,853307,Science Of Sleep - Version B,0.000,False
624833,False,853308,情事：嫉妒陷阱,0.000,False
624834,False,853310,Real Fantasies,0.000,False


In [6]:
# Getting Full info for all these movies
info_list = []

for m_id in tqdm.tqdm(list(all_movies.id)):
    r = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(m_id, api_key))
    info_list.append(json.loads(r.content.decode(encoding="ascii", errors="ignore")))

100%|██████████| 25000/25000 [41:29<00:00, 10.04it/s]  


In [7]:
# Getting Cast List for all these movies
credit_list = []

for m_id in tqdm.tqdm(list(all_movies.id)):
    r = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key={}'.format(m_id, api_key))
    credit_list.append(json.loads(r.content.decode(encoding="ascii", errors="ignore")))

100%|██████████| 25000/25000 [41:43<00:00,  9.99it/s]  


In [9]:
# check if ratings in here

In [10]:
# So we can come back and get the names of all these actors later if we need to. Really ids are totally fine for this

In [16]:
movie_records = []
movie_features = ['adult','id','popularity','original_title','original_langue',
                  'release_date','revenue', 'vote_average', 'vote_count', 'budget', 'imdb_id']

for inf in info_list:
    try:
        movie_records.append([inf.get(key) for key in movie_features])
    except:
        print('failed on: {}'.format(inf['original_title']))

movie_df = pd.DataFrame(movie_records, columns=movie_features)

In [17]:
# So this doesn't include the movieid, so that'll be important to incldue zip in the id too

cast_features = ['gender', 'id', 'name', 'order']

cast_records = []
for m_id, cast in zip(list(all_movies.id), credit_list):
    # Sometimes you get some crap status codes, I could handle this better but meh
    try: 
        for char in cast['cast']:
            tmp_list = []
            cst_list = [char.get(key) for key in cast_features]
            tmp_list.extend(cst_list)
            tmp_list.append(m_id)
            cast_records.append(tmp_list)
    except:
        pass
    
cast_df = pd.DataFrame(cast_records, columns=cast_features+['m_id'])

In [18]:
# So this doesn't include the movieid, so that'll be important to incldue zip in the id too

cast_features = ['gender', 'id', 'name', 'order']

cast_records = []
for m_id, cast in zip(list(all_movies.id), credit_list):
    # Sometimes you get some crap status codes, I could handle this better but meh
    try: 
        for char in cast['cast']:
            tmp_list = []
            cst_list = [char.get(key) for key in cast_features]
            tmp_list.extend(cst_list)
            tmp_list.append(m_id)
            cast_records.append(tmp_list)
    except:
        pass
    
cast_df = pd.DataFrame(cast_records, columns=cast_features+['m_id'])

crew_features = ['gender', 'department', 'id', 'job', 'name']

crew_records = []
for m_id, cast in zip(list(all_movies.id), credit_list):
    # Sometimes you get some crap status codes, I could handle this better but meh
    try: 
        for char in cast['crew']:
            tmp_list = []
            crw_list = [char.get(key) for key in crew_features]
            tmp_list.extend(crw_list)
            tmp_list.append(m_id)
            crew_records.append(tmp_list)
    except:
        pass
    
crew_df = pd.DataFrame(crew_records, columns=crew_features+['m_id'])

In [19]:
cast_df.to_csv('cast_df.csv')
crew_df.to_csv('crew_df.csv')
movie_df.to_csv('movie_df.csv')

In [22]:
# Imdb rating dump available at https://datasets.imdbws.com/
# The fucking file is named data.tsv? Who the hell names a data file "data"
# Whatever, they still give it to me for free so I'll take it
ratings = pd.read_csv("data.tsv",  sep='\t')
ratings.to_csv('ratings.csv')

In [23]:
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1739
1,tt0000002,6.0,218
2,tt0000003,6.5,1502
3,tt0000004,6.1,134
4,tt0000005,6.2,2307
...,...,...,...
1172617,tt9916682,5.6,5
1172618,tt9916690,6.6,5
1172619,tt9916720,5.4,115
1172620,tt9916766,6.9,18
